In [1]:
clean_up = True
%run stdPackages.ipynb
ws = gams.GamsWorkspace(working_directory=d['work']) # specify where you want to run the GAMS models from (here the repository referred to in d['work'])

The file _gams_py_gdb0.gdx is still active and was not deleted.
The file _gams_py_gdb1.gdx is still active and was not deleted.
The file _gams_py_gdb2.gdx is still active and was not deleted.
The file _gams_py_gdb3.gdx is still active and was not deleted.
The file _gams_py_gdb4.gdx is still active and was not deleted.



## Settings/data

In [2]:
t = 2018
name = 'm'
db_IO = GpyDB(pickle_path = os.path.join(d['data'], f'{name}_{t}')) # load IO database named IO_name
with open(f"{d['data']}\\glob_{t}","rb") as file: # load global settings anmed glob_name
    glob=pickle.load(file)

In [3]:
def loadFromPickle(x):
    with open(os.path.join(d['data'],x), "rb") as file:
        return pickle.load(file)

Add time index to the IO data - but only use the year $t_0$:

In [4]:
def addT(symbol, t):
    return adjMultiIndex.bc(symbol, t).reorder_levels(['t']+symbol.index.names if 't' not in symbol.index.names else symbol.index.names)
[db_IO.__setitem__(k, addT(db_IO.get(k), glob.db['t0'].vals)) for k in db_IO.getTypes(['variable','scalar_variable'])];

Load modules:

In [5]:
modules = [f"{name}_{k}" for k in ('G','HH','itory','I','P','Trade')]
GE = gmsPyModels.GmsPython(name = f"{name}_GE_{t}", glob=glob, m = [loadFromPickle(n) for n in modules], s_kwargs = {'ws': ws})
robust.robust_merge_dbs(GE.s.db, db_IO, priority = 'first')

### Calibrate model

Add equilibrium module:

In [6]:
mEqui = mEquilibrium.Equilibrium(name = name, db_IO = db_IO, glob = glob)
GE.addModule(mEqui, merge_s = True)

Set up model, write gams code, and calibrate model to IO data:

In [7]:
GE.compile()
GE.write()
mCalib, cp = GE.sneakyCalib(db_IO, ws = ws, loop_kwargs = {'n': 10}, **{'cns': 'CONOPT4'})

Export solution:

In [8]:
robust.robust_merge_dbs(GE.s.db, mCalib.out_db.getTypes(['variable','scalar_variable']), priority = 'second')
GE.s.setstate('B')

In [9]:
with open(os.path.join(d['data'], GE.name), "wb") as file:
    pickle.dump(GE,file)